import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import os
import s3fs
import boto3
from io import BytesIO as bo
import gc

Calculate Date Variables

In [33]:
# Variable Calcuation Section :

#Since data is received on Monday
#Getting this weeks monday from today's date.

##############################
#FOR CUR_PROC_WK, TODAY, PRE_PROC_WK,CUR_WK,PRE_WK
this_day = datetime.today()

# #####
# # for testing :
# this_day = this_day - timedelta(days=7) # REMOVE ON REGULAR RUNS
# #####

days_to_monday = (this_day.weekday() - 0) % 7
monday = this_day - timedelta(days=days_to_monday)
CUR_PROC_WK = monday.strftime("%Y%m%d")  #Converting to desired string format
TODAY = monday  #Keeping datetime format
PRE_PROC_WK0 = monday - timedelta(days=7)
PRE_PROC_WK = str(PRE_PROC_WK0.year) + str(PRE_PROC_WK0.month).zfill(2) + str(PRE_PROC_WK0.day).zfill(2)

#xponent data week, even in month closing week, still use FRIDAY date;
CUR_WK0 = monday - timedelta(days=17)
CUR_WK = str(CUR_WK0.year) + str(CUR_WK0.month).zfill(2) + str(CUR_WK0.day).zfill(2)
DATA_DATE_WEEKLY = f"{CUR_WK0.year}-{CUR_WK0.month:02d}-{CUR_WK0.day:02d}"
YR = CUR_WK0.year

#weekly events and calls data date - NO two weeks delay(FRIDAY);
EVTS_DATE_WEEKLY0 = monday - timedelta(days=3)
EVTS_DATE_WEEKLY = f"{EVTS_DATE_WEEKLY0.year}-{EVTS_DATE_WEEKLY0.month:02d}-{EVTS_DATE_WEEKLY0.day:02d}"

#previous data week - FRIDAY
PRE_WK0 = monday - timedelta(days=24)
PRE_WK = PRE_WK0.strftime("%Y%m%d")

DATA_DATE_CALENDAR = f"{monday.year}-{monday.month:02d}-{monday.day:02d}"
RECORD_END_DATE = pd.PeriodIndex([this_day], freq='Q').end_time[0].strftime('%Y-%m-%d')
##############################
QTR = f'{CUR_WK0.year}Q{(CUR_WK0.month-1)//3+1}'
QTR_NET = f"Q{((CUR_WK0.month - 1) // 3 + 1)}"
CUT_OFF_net = ((pd.PeriodIndex([CUR_WK0], freq='Q')) + 1).start_time[0].strftime('%Y-%m-%d')
##############################
month_end_date = CUR_WK0.replace(day=1) - timedelta(days=1) #This hold the month ending date for the month before current xpn date
# Month closing week
if 0 <= (CUR_WK0 - month_end_date).days < 7:
    # Xponent data month, in month closing week
    # Xponent data date or last day of the previous month (if month closing) - FRIDAY
    DATA_DATE_MONTHLY = str(month_end_date.year) + '-' + str(month_end_date.month).zfill(2) + '-' + str(month_end_date.day).zfill(2)
else:
    # Xponent data month, not in month closing week
    # Xponent data date or last day of the previous month (if month closing) - FRIDAY
    DATA_DATE_MONTHLY = DATA_DATE_WEEKLY

##############################

In [34]:
# CUR_PROC_WK2 = '20240702'


In [35]:
# Grabbing IW Release File 
s3 = s3fs.S3FileSystem()
bucket_list = s3.listdir(f'vortex-staging-a65ced90/PYADM/raw/{CUR_PROC_WK}/inbound/')
for file in bucket_list:
    if (file['Key'].__contains__(f"IRWD_RELEASE_WKLY_{CUR_PROC_WK}")):
        release_file = file['Key']

# Reading Release File -
s3_client = boto3.client('s3')
bucket = release_file.split('/')[0]
file_key = release_file.split('/',1)[1]

adm = pd.read_csv(f's3://{bucket}/{file_key}',sep='|')
adm1 = adm[CUR_PROC_WK].tolist()

for item in adm1:
    if item.startswith('P_OUT_110_HCP-ZIP-DEFAULT_'):
        HCPZD_TIMESTAMP = item.split('_')[-1].split('.')[0]
    elif item.startswith('P_OUT_110_HCP_'):
        HCP_TIMESTAMP = item.split('_')[-1].split('.')[0]

Library Names and File Paths :

In [36]:
raw_path = f'PYADM/raw/{CUR_PROC_WK}/inbound/'
qtrspec = f'PYADM/quaterly/{QTR}/reference/'
curwk = f'PYADM/raw/{CUR_PROC_WK}/dataframes/'
ref_pwek = f'PYADM/weekly/archive/{PRE_WK}/reference/'
ref_week = 'PYADM/weekly/staging/reference/'
ref_month = 'PYADM/monthly/staging/reference/'
ZIP = f'PYADM/quaterly/{QTR}/geography/'
jami = f'PYADM/reference/{QTR}/'

Importing ZIP Default Files-

In [37]:
%%time
temp_CUSTOMER_ZD = pd.read_csv(
    f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT_{HCPZD_TIMESTAMP}.TXT',
    delimiter='|',
    dtype={1:'Int64',37:'Int64',38:'Int64',32:str,17:str,29:str,20:str,18:str,26:str,21:str,15:str,23:str,25:str,24:str,22:str,14:str},
    parse_dates=[31]
)
temp_CUSTOMER_ZD.rename(columns={'win_i_id': 'IronwoodWinnerID'}, inplace=True)

temp_CUSTOMERFLAG_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-FLG_{HCPZD_TIMESTAMP}.TXT', delimiter='|')

temp_ADDRESS_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-ADDR_{HCPZD_TIMESTAMP}.TXT', delimiter='|', dtype={1:str,11:str,10: str,6:str,12:str})

temp_ADDRESSFLAG_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-ADDR-FLG_{HCPZD_TIMESTAMP}.TXT', delimiter='|')
temp_ADDRESSFLAG_ZD.rename(columns={'AddressSourceID': 'AddressSourceId'}, inplace=True)

temp_STATELICENSE_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-STATE-LIC_{HCPZD_TIMESTAMP}.TXT', delimiter='|')

temp_COMMUNICATION_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-COMM_{HCPZD_TIMESTAMP}.TXT', delimiter='|')

temp_SPECIALTY_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-SPECIALTIES_{HCPZD_TIMESTAMP}.TXT', delimiter='|', dtype = {4:str,5:str,1:str})

temp_DEA_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-DEA_{HCPZD_TIMESTAMP}.TXT', delimiter='|')

temp_IDENTIFIER_ZD = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ZIP-DEFAULT-IDENTIFIERS_{HCPZD_TIMESTAMP}.TXT', delimiter='|', dtype={'IID':'Int64'})

CPU times: user 178 ms, sys: 4.98 ms, total: 183 ms
Wall time: 776 ms


Import HCP Files-

In [71]:
%%time
# GET S3 OBJ function was faster by 10 seconds!!!
temp_CUSTOMER1 = pd.read_csv(
    f's3://{bucket}/{raw_path}P_OUT_110_HCP_{HCP_TIMESTAMP}.TXT',
    delimiter='|',
    dtype={1:'Int64',37:'Int64',38:'Int64',
          32:str,6:str,17:str,29:str,20:str,18:str,26:str,21:str,15:str,23:str,25:str,24:str,22:str,14:str},
    parse_dates=[31]
)

temp_CUSTOMER1.rename(columns={'win_i_id': 'IronwoodWinnerID'}, inplace=True)
qc_before = len(temp_CUSTOMER1)
# DEDUP on 'IID'
temp_CUSTOMER1.drop_duplicates(subset='IID', keep='first', inplace=True)
qc_after = len(temp_CUSTOMER1)
print(" Dropped Rows from Dedup of Customer Component : ",qc_before-qc_after)

temp_CUSTOMERFLAG1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-FLG_{HCP_TIMESTAMP}.TXT', delimiter='|')

temp_ADDRESS1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ADDR_{HCP_TIMESTAMP}.TXT', delimiter='|', dtype={1:str,11:str,10: str,6:str,12:str}, on_bad_lines='warn')

temp_ADDRESSFLAG1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-ADDR-FLG_{HCP_TIMESTAMP}.TXT', delimiter='|')
temp_ADDRESSFLAG1.rename(columns={'AddressSourceID': 'AddressSourceId'}, inplace=True)

temp_STATELICENSE1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-STATE-LIC_{HCP_TIMESTAMP}.TXT', delimiter='|')

temp_COMMUNICATION1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-COMM_{HCP_TIMESTAMP}.TXT', delimiter='|')

temp_SPECIALTY1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-SPECIALTIES_{HCP_TIMESTAMP}.TXT', delimiter='|', dtype = {4:str,5:str,1:str})

temp_DEA1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-DEA_{HCP_TIMESTAMP}.TXT', delimiter='|')

temp_IDENTIFIER1 = pd.read_csv(f's3://{bucket}/{raw_path}P_OUT_110_HCP-IDENTIFIERS_{HCP_TIMESTAMP}.TXT', delimiter='|', dtype={'IID':'Int64',1:str,2:str},
                               converters={'IdentifierValue': lambda x: str(x) if x != 'nan' else ''}
                              )


 Dropped Rows from Dedup of Customer Component :  0
CPU times: user 9.58 s, sys: 747 ms, total: 10.3 s
Wall time: 27.5 s


##### FIX FOR MERGING ADDRESS AND FLAG

In [80]:
%%time
# *Dedup Address Components first -;
# THIS SHOULD HAVE ZERO DUPS

qc_before = len(temp_ADDRESS_ZD)
temp_ADDRESS_ZD.drop_duplicates(subset=['IID', 'AddressSourceId'], keep='first', inplace=True)
# Adding Sorting Step
temp_ADDRESS_ZD.sort_values('AddressSourceId',inplace=True,ignore_index=True)
qc_after = len(temp_ADDRESS_ZD)
print(" temp_ADDRESS_ZD dropped : ",qc_before-qc_after)

qc_before = len(temp_ADDRESSFLAG_ZD)
temp_ADDRESSFLAG_ZD.drop_duplicates(subset=['IID', 'AddressSourceId','AddressFlagType'], keep='first', inplace=True)
qc_after = len(temp_ADDRESSFLAG_ZD)
print(" temp_ADDRESSFLAG_ZD dropped : ",qc_before-qc_after)

qc_before = len(temp_ADDRESS1)
temp_ADDRESS1.drop_duplicates(subset=['IID', 'AddressSourceId'], keep='first', inplace=True)
# Adding Sorting Step
temp_ADDRESS1.sort_values('AddressSourceId',inplace=True,ignore_index=True)
qc_after = len(temp_ADDRESS1)
print(" temp_ADDRESS1 dropped : ",qc_before-qc_after)

qc_before = len(temp_ADDRESSFLAG1)
temp_ADDRESSFLAG1.drop_duplicates(subset=['IID', 'AddressSourceId','AddressFlagType'], keep='first', inplace=True)
qc_after = len(temp_ADDRESSFLAG1)
print(" temp_ADDRESSFLAG1 dropped : ",qc_before-qc_after)

 temp_ADDRESS_ZD dropped :  0
 temp_ADDRESSFLAG_ZD dropped :  0
 temp_ADDRESS1 dropped :  0
 temp_ADDRESSFLAG1 dropped :  0
CPU times: user 3.54 s, sys: 45 ms, total: 3.58 s
Wall time: 4.11 s


In [81]:
# Merging Addr with Addr Flag ;
temp_ADDRESS_ZD_F = pd.merge(temp_ADDRESS_ZD, temp_ADDRESSFLAG_ZD[['IID', 'AddressSourceId','AddressFlagType']],
                             how = 'left',
                             left_on = ['IID', 'AddressSourceId'],
                             right_on = ['IID', 'AddressSourceId'])
temp_ADDRESS_ZD=temp_ADDRESS_ZD_F.copy()

temp_ADDRESS1_F = pd.merge(temp_ADDRESS1, temp_ADDRESSFLAG1[['IID', 'AddressSourceId','AddressFlagType']],
                             how = 'left',
                             left_on = ['IID', 'AddressSourceId'],
                             right_on = ['IID', 'AddressSourceId'])
temp_ADDRESS1=temp_ADDRESS1_F.copy()

##### JOIN ALL PARTITIONS AND ZIP DEFAULT FILE

In [82]:
%%time
def comb_hcp(DF_HCP,DF_ZD):
    combined_df = pd.concat([DF_HCP, DF_ZD], ignore_index=True)
    return(combined_df)

curwk_CUSTOMER = comb_hcp(temp_CUSTOMER1,temp_CUSTOMER_ZD) 
curwk_CUSTOMERFLAG = comb_hcp(temp_CUSTOMERFLAG1,temp_CUSTOMERFLAG_ZD) 
curwk_ADDRESS = comb_hcp(temp_ADDRESS1,temp_ADDRESS_ZD) 
curwk_STATELICENSE = comb_hcp(temp_STATELICENSE1,temp_STATELICENSE_ZD)
curwk_COMMUNICATION = comb_hcp(temp_COMMUNICATION1,temp_COMMUNICATION_ZD) 
curwk_SPECIALTY = comb_hcp(temp_SPECIALTY1,temp_SPECIALTY_ZD) 
curwk_DEA = comb_hcp(temp_DEA1,temp_DEA_ZD) 
curwk_IDENTIFIER = comb_hcp(temp_IDENTIFIER1,temp_IDENTIFIER_ZD) 

CPU times: user 452 ms, sys: 4.12 ms, total: 456 ms
Wall time: 559 ms


<timed exec>:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


##### CREATE CURRENT WEEK CUSTOMER_MASTER

In [83]:
# Date related columns from Customer Component need to be changed from text to datetime format so that filtering can be done later
# Added this because 9999-12-31 is not supported by the datetime converter
# So active Customer will now be designated by 2262-04-11 instead as its winthin supported date range

curwk_CUSTOMER.loc[curwk_CUSTOMER['CustomerEffectiveEndDate'] == '9999-12-31', 'CustomerEffectiveEndDate'] = '2262-04-11'

# Date Column type change for Identifier -
curwk_CUSTOMER['CustomerEffectiveStartDate'] = pd.to_datetime(curwk_CUSTOMER['CustomerEffectiveStartDate'], format='%Y-%m-%d', errors='coerce')
curwk_CUSTOMER['CustomerEffectiveEndDate'] = pd.to_datetime(curwk_CUSTOMER['CustomerEffectiveEndDate'], format='%Y-%m-%d', errors='coerce')


In [84]:
# Dedup CUSTOMER component
qc_before = len(curwk_CUSTOMER)
temp_CUSTOMER = curwk_CUSTOMER.sort_values(by='IID').drop_duplicates(subset='IID', keep='first')
temp_CUSTDUP = curwk_CUSTOMER[curwk_CUSTOMER.duplicated(subset='IID', keep=False)]
qc_after = len(temp_CUSTOMER)
print(" curwk_CUSTOMER dropped : ",qc_before-qc_after)


 curwk_CUSTOMER dropped :  0


In [85]:
# Date related columns from Address Component need to be changed from text to datetime format so that filtering can be done
# Added this because 9999-12-31 is not supported by the datetime converter
# So active addresses will now be designated by 2262-04-11 instead as its winthin supported date range
curwk_ADDRESS.loc[curwk_ADDRESS['AddressEffectiveEndDate'] == '9999-12-31', 'AddressEffectiveEndDate'] = '2262-04-11'

# Date Column type change for address - 
curwk_ADDRESS['AddressEffectiveStartDate'] = pd.to_datetime(curwk_ADDRESS['AddressEffectiveStartDate'], format='%Y-%m-%d', errors='coerce')
curwk_ADDRESS['AddressEffectiveEndDate'] = pd.to_datetime(curwk_ADDRESS['AddressEffectiveEndDate'], format='%Y-%m-%d', errors='coerce')


##### Dedup and CLean ADDRESS

In [86]:
# creating empty dataframes first
temp_ADDRESSEND = pd.DataFrame() #this will store all the addresses that failed the filters
temp_ADDRESS = pd.DataFrame() #final filterted result

# Defining the conditions -
condition1 = curwk_ADDRESS['AddressEffectiveEndDate'] < TODAY
condition2 = curwk_ADDRESS['AddressStatusCode'] == 'Inactive'
condition3 = curwk_ADDRESS['AddressEffectiveEndDate'].isna()  # Additional condition for NaT values

# applying filter - 
temp_ADDRESSEND = curwk_ADDRESS[(condition1 | condition2 | condition3)].copy()
temp_ADDRESS = curwk_ADDRESS[~(condition1 | condition2 | condition3)].copy() #ive simply used negagation '~' here to flip the condtions here

#qc 
print("There were ",len(curwk_ADDRESS),"Records in curwk_ADDRESS")
print("There ARE ",len(temp_ADDRESS),"Records in temp_ADDRESS")
print("There ARE ",len(temp_ADDRESSEND),"Records in temp_ADDRESSEND")

There were  916545 Records in curwk_ADDRESS
There ARE  915394 Records in temp_ADDRESS
There ARE  1151 Records in temp_ADDRESSEND


#### 2.1: Check each IID only has one BOB address -

In [87]:
# # QC Count the number of addresses with AddressFlagType = 'BOB' for each hcp
# this approach is no longer working - 
# bob_count = temp_ADDRESS.groupby('IID')['AddressFlagType'].apply(lambda x: (x == 'BOB').sum()).reset_index()
# bob_count.rename(columns={'AddressFlagType': 'BOB Count'}, inplace=True)

# bob_count_freq_table = bob_count['BOB Count'].value_counts().sort_index().reset_index()
# bob_count_freq_table.rename(columns={'index': 'BOB Count', 'BOB Count': 'count'}, inplace=True)
# bob_count_freq_table = bob_count_freq_table.reindex(range(len(bob_count_freq_table)), fill_value=0)

# bob_count_freq_table.columns.values[0] = 'BOB_Count'
# bob_count_freq_table.columns.values[1] = 'No_of_HCPs'

# print("Frequency Table for BOB Count:")
# print(bob_count_freq_table.to_string(index=False))

# this code is calculating the number of times ‘BOB’ appears in the AddressFlagType column for each unique IID, and then creating a 
# frequency table of these counts. The frequency table shows how many IIDs have each possible count of ‘BOB’.

bob_count = temp_ADDRESS[['IID','AddressFlagType']].copy()
bob_count['HCPs'] = bob_count['AddressFlagType'].apply(lambda x: 1 if x == 'BOB' else 0)
bob_count_grouped = bob_count.groupby('IID')['HCPs'].sum()

# Frequency table
frequency_table = bob_count_grouped.value_counts().sort_index()
print("Frequency Table for BOB Count:")
print(frequency_table)

Frequency Table for BOB Count:
HCPs
0      8887
1    131716
Name: count, dtype: int64


### BOB filtering notes : 
#### from my understanding of the sas codes we have 5 cases to consider - 
* case 1- Only 1 Address, and Count of BOB flag = 1
- case 2- More than 1 Address, and Count of BOB flag = 1 [in this case we only keep the addr with bob flag]
- case 3- More than 1 Address, and Count of BOB flag = 0 / Null [I.e no BOB flag] , [ In this case we keep the first obs]
- case 4- Only 1 Address, and Count of BOB flag = 0 / Null [I.e no BOB flag], [In this case the assumption is the only avaialble address is the best address so we keep it]
- case 5- Count of BOB flag grater than 1 , Outlier case , this implys there are multiple addresses with BOB flag Droppinging this case for now
Line  
*We were NOT assinging BOB to cases with only one addr and no bob flag , contrary to previous understanding*

In [88]:
# BOB Clean and Dedup by IID
temp_test1 = temp_ADDRESS.copy() #creating a new df to do processing for BOB flag filtering

#adding 2 flags to temp_test1 for calculation and filtering - 
Co_Addr = temp_test1.groupby('IID').size()
Co_Addr.name = 'Co_Addr' #count of addresses [ we can use hub_id to count number of addresses for each hub_id]

Co_BOB = temp_test1[temp_test1['AddressFlagType'] == 'BOB'].groupby('IID').size()
Co_BOB.name = 'Co_BOB'

temp_test1 = temp_test1.merge(Co_Addr, on='IID', how='left')
temp_test1 = temp_test1.merge(Co_BOB, on='IID', how='left')
#qc
#temp_test1.shape

# following are dataframes made by filtering temp_test1
# we filter it to create sub components to cater each condition , then we merge them back together

temp_test2 = temp_test1[temp_test1['Co_Addr'] == 1].copy() #case 1 , 4
temp_test2.reset_index(drop=True, inplace=True) #as long as there is one address keeping it , BOB or not
#print(temp_test2.shape)

temp_test3 = temp_test1[(temp_test1['Co_Addr'] > 1) & (temp_test1['Co_BOB'].isnull() | (temp_test1['Co_BOB'] == 0))].copy() #case 3
temp_test3.reset_index(drop=True, inplace=True)
temp_test3.drop_duplicates(subset='IID', keep='first', inplace=True) #only keeping first occurance
#print(temp_test3.shape)

temp_test4 = temp_test1[(temp_test1['Co_Addr'] > 1) & (temp_test1['Co_BOB'] == 1)].copy() #case 2
temp_test4 = temp_test4[temp_test4['AddressFlagType'] == 'BOB'].copy()
#print(temp_test4.shape) # only keeping instances with BOB

temp_test5 = temp_test1[temp_test1['Co_BOB'] > 1].copy() #case 5 [outlier scenario]
#print(temp_test5.shape)

# Combining Components here - 
# Concatenate temp_test2, temp_test3, and temp_test4 into temp_Addr_all
temp_Addr_all = pd.concat([temp_test2, temp_test3, temp_test4], ignore_index=True)
#print(temp_Addr_all.shape)

#Deduping final Filtered Dataset by IID [or IID in our case] -

# Keep only the first occurrence of each IID and drop the duplicates
temp_ADDRESSNODUP = temp_Addr_all.drop_duplicates(subset='IID', keep='first')
temp_ADDRESSNODUP = temp_ADDRESSNODUP.drop(columns=['Co_Addr', 'Co_BOB']) # FINAL DATASET READY

# Save the duplicates in a separate DataFrame [This may be a redundant practice carried over from sas]
temp_AddressDup = temp_Addr_all[temp_Addr_all.duplicated(subset='IID')]

#### Dedup and Clean SPECIALTY -

In [89]:
curwk_SPECIALTY.loc[curwk_SPECIALTY['SpecialtyEffectiveEndDate'] == '9999-12-31', 'SpecialtyEffectiveEndDate'] = '2262-04-11'
# Date Column type change for address -
curwk_SPECIALTY['SpecialtyEffectiveStartDate'] = pd.to_datetime(curwk_SPECIALTY['SpecialtyEffectiveStartDate'], format='%Y-%m-%d', errors='coerce')
curwk_SPECIALTY['SpecialtyEffectiveEndDate'] = pd.to_datetime(curwk_SPECIALTY['SpecialtyEffectiveEndDate'], format='%Y-%m-%d', errors='coerce')

In [90]:
temp_SPECIALTY = curwk_SPECIALTY.copy()
qc_before = len(temp_SPECIALTY)
temp_SPECSORT = temp_SPECIALTY.sort_values(by = 'IID').drop_duplicates(subset='IID', keep='first')
temp_SPECDUP = temp_SPECIALTY[temp_SPECIALTY.duplicated(subset='IID', keep=False)]
qc_after = len(temp_SPECSORT)
print(" IID dropped : ",qc_before-qc_after)

 IID dropped :  0


In [91]:
# new Specialty Check
qc_before = len(temp_SPECIALTY)
temp_SPECIALTYSORT = temp_SPECIALTY.sort_values(by = 'SpecialtyCode').drop_duplicates(subset='SpecialtyCode', keep='first')
temp_SPECIALTYDUP = temp_SPECIALTY[temp_SPECIALTY.duplicated(subset='SpecialtyCode', keep=False)]
qc_after = len(temp_SPECSORT)
print(" SpecialtyCode dropped : ",qc_before-qc_after)

 SpecialtyCode dropped :  0


In [92]:
# new spec check
# Fetching Data from stored location - 
qtrspec_SPEC_INCL_LIN = pd.read_parquet(f's3://{bucket}/{qtrspec}qtrspec_SPEC_INCL_LIN.parquet')

# Merging with qaterly file to find new specs -
temp_CHECK_SPECIALTY = pd.merge(temp_SPECIALTYSORT, qtrspec_SPEC_INCL_LIN, how='left', left_on='SpecialtyCode', right_on='SPECIALTY_CD', indicator=True)
# Filter the merged_data using conditions A AND NOT B
temp_CHECK_SPECIALTY = temp_CHECK_SPECIALTY[(temp_CHECK_SPECIALTY['_merge'] == 'left_only')]
# Drop the _merge column as its not needed
temp_CHECK_SPECIALTY.drop('_merge', axis=1, inplace=True)
temp_CHECK_SPECIALTY = temp_CHECK_SPECIALTY[['SpecialtyCode']]

#Print Results -
if temp_CHECK_SPECIALTY.empty:
    print("No new specialty")
else:
    print("2.3 List of Spec_Cds not avialable in qtrspec_SPEC_INCL_LIN : ")
    print(temp_CHECK_SPECIALTY['SpecialtyCode'].tolist())

2.3 List of Spec_Cds not avialable in qtrspec_SPEC_INCL_LIN : 
['ASO', 'DAP', 'GLAU', 'OID', 'TY', 'URPS', 'VEC', 'VEN', 'VER', 'VES']


##### process IDENTIFIERS-

In [104]:
#fix max end date -
curwk_IDENTIFIER.loc[curwk_IDENTIFIER['IdentifierEffectiveEndDate'] == '9999-12-31', 'IdentifierEffectiveEndDate'] = '2262-04-11'
curwk_IDENTIFIER['IdentifierEffectiveStartDate'] = pd.to_datetime(curwk_IDENTIFIER['IdentifierEffectiveStartDate'], format='%Y-%m-%d', errors='coerce')
curwk_IDENTIFIER['IdentifierEffectiveEndDate'] = pd.to_datetime(curwk_IDENTIFIER['IdentifierEffectiveEndDate'], format='%Y-%m-%d', errors='coerce')

In [106]:
#NPI
NPI_CUR_PROC_WK = pd.DataFrame()
condition1 = curwk_IDENTIFIER['IdentifierType'] == 'NPI'
condition2 = curwk_IDENTIFIER['IdentifierEffectiveEndDate'] > TODAY
condition3 = curwk_IDENTIFIER['IdentifierValue'] != ""
condition4 = curwk_IDENTIFIER['IdentifierEffectiveEndDate'].isna()

NPI_CUR_PROC_WK = curwk_IDENTIFIER[(condition1 & condition2 & condition3 & ~(condition4))].copy()
NPI_CUR_PROC_WK.drop_duplicates(subset='IID', keep='first', inplace=True)

NPI_CUR_PROC_WK.rename(columns={'IdentifierValue': 'NPI_ID','IdentifierEffectiveStartDate':'NPI_StartDate','IdentifierEffectiveEndDate':'NPI_EndDate'}, inplace=True)
NPI_CUR_PROC_WK = NPI_CUR_PROC_WK[['IID','NPI_ID','NPI_StartDate','NPI_EndDate']]

#Note - Historically Npi Datasets from all weeks are combined together.

In [107]:
#ME
temp_IDENTIFIER_ME = pd.DataFrame()
condition1 = curwk_IDENTIFIER['IdentifierType'] == 'ME'
condition2 = curwk_IDENTIFIER['IdentifierEffectiveEndDate'] > TODAY
condition3 = curwk_IDENTIFIER['IdentifierEffectiveEndDate'].isna()
temp_IDENTIFIER_ME = curwk_IDENTIFIER[(condition1 & condition2 & ~(condition3))].copy()

temp_IDENTIFIERSORT = temp_IDENTIFIER_ME.copy()
temp_IDENTIFIERSORT.drop_duplicates(subset='IID', keep='first', inplace=True)
temp_IDENTIFIERSORT.reset_index(drop=True, inplace=True)

In [108]:
# Saw Cases of Records where both ME ID and NPI ID where aviailable for the HCP
temp_ME_NPI = temp_IDENTIFIERSORT.merge(NPI_CUR_PROC_WK, how='outer', on='IID')

In [109]:
#Taxonomy
temp_IDENTIFIER_TAXONOMY = pd.DataFrame()
condition1 = curwk_IDENTIFIER['IdentifierType'] == 'TAXONOMY'
condition2 = curwk_IDENTIFIER['IdentifierEffectiveEndDate'] > TODAY
condition3 = curwk_IDENTIFIER['IdentifierEffectiveEndDate'].isna()
temp_IDENTIFIER_TAXONOMY = curwk_IDENTIFIER[(condition1 & condition2 & ~(condition3))].copy()

temp_IDENTIFIER2SORT = temp_IDENTIFIER_TAXONOMY.copy()
temp_IDENTIFIER2SORT.drop_duplicates(subset='IID', keep='first', inplace=True)
temp_IDENTIFIER2SORT.reset_index(drop=True, inplace=True)

temp_IDENTIFIER2SORT.rename(columns={'IdentifierValue': 'TAXONOMYID'}, inplace=True)
temp_IDENTIFIER2SORT = temp_IDENTIFIER2SORT[['IID','TAXONOMYID']]

In [110]:
temp_ME_NPI_TAX = temp_ME_NPI.merge(temp_IDENTIFIER2SORT, how='outer', on='IID')

In [115]:
#temp_IDENTIFIERS = pd.concat([NPI_CUR_PROC_WK,temp_IDENTIFIERSORT,temp_IDENTIFIER2SORT],axis = 0)
#temp_IDENTIFIERS = pd.concat([temp_IDENTIFIERSORT,temp_IDENTIFIER2SORT],axis = 0)
temp_IDENTIFIERS=temp_ME_NPI_TAX.copy()

### Creating Final Weekly Master Profile Delta -

In [123]:
#Create Weekly Delta Customer Master
curwk_CUSTOMER_MASTER = pd.DataFrame()

# I am assuming that for the case of overlapping columns , we want to keep stuff from Customer Component only
# Perform the merge operation ( second line is drop the overlapping columns from right dataset)
curwk_CUSTOMER_MASTER = pd.merge(temp_CUSTOMER, temp_ADDRESSNODUP, on='IID', how='left',suffixes=(None, '_y'))
curwk_CUSTOMER_MASTER = curwk_CUSTOMER_MASTER[[col for col in curwk_CUSTOMER_MASTER.columns if not col.endswith('_y')]]

curwk_CUSTOMER_MASTER = pd.merge(curwk_CUSTOMER_MASTER, temp_SPECSORT, on='IID', how='left',suffixes=(None, '_y'))
curwk_CUSTOMER_MASTER = curwk_CUSTOMER_MASTER[[col for col in curwk_CUSTOMER_MASTER.columns if not col.endswith('_y')]]

curwk_CUSTOMER_MASTER = pd.merge(curwk_CUSTOMER_MASTER, temp_IDENTIFIERS, on='IID', how='left',suffixes=(None, '_y'))
curwk_CUSTOMER_MASTER = curwk_CUSTOMER_MASTER[[col for col in curwk_CUSTOMER_MASTER.columns if not col.endswith('_y')]]

In [124]:
#Final Changes - 

# Rename columns and change data types
curwk_CUSTOMER_MASTER = curwk_CUSTOMER_MASTER.rename(columns={'PostalCode': 'ZIP', 'SecondaryID': 'IMSID'})
curwk_CUSTOMER_MASTER['IMSID'] = curwk_CUSTOMER_MASTER['IMSID'].astype(str)

#Coluns that need to be dropped ? :
curwk_CUSTOMER_MASTER.drop(columns={'TransactionID' , 'PartnerID' , 'PartnerLoserID' ,'PartnerWinnerID', 'PartnerName'}, axis=1, inplace=True)


In [125]:
curwk_CUSTOMER_MASTER.shape

(141739, 70)

In [126]:
# Temp fix for Identifer 
curwk_CUSTOMER_MASTER = curwk_CUSTOMER_MASTER.astype({'IdentifierValue': str})
#print(curwk_CUSTOMER_MASTER['IdentifierValue'].apply(type).value_counts())

In [129]:
curwk_CUSTOMER_MASTER.to_parquet(f's3://{bucket}/{curwk}curwk_CUSTOMER_MASTER.parquet')
curwk_CUSTOMER_MASTER.shape
# #weekly master profile delta ready

(141739, 70)

### 3 - Profiles Delta- Create HCP Universe, Adds, Updates, Merges

In [130]:
#ZIP length check
temp_ZIPL_CHECK = curwk_CUSTOMER_MASTER.copy()
temp_ZIPL_CHECK['ZIPL'] = temp_ZIPL_CHECK['ZIP'].str.len() #adding new column for zip length
all_rec = len(temp_ZIPL_CHECK)
temp_ZIPL_CHECK = temp_ZIPL_CHECK[temp_ZIPL_CHECK['ZIPL'] != 5].copy() #filtering records where zip length is not 5 [non compliant zip code]
invalid_zip = len(temp_ZIPL_CHECK)
temp_ZIPL_CHECK = temp_ZIPL_CHECK[temp_ZIPL_CHECK['CustomerStatusCode'] == 'Active'].copy() #[then filtering out how many of them are for active hcps]
act_invalid_zip = len(temp_ZIPL_CHECK)
print("--- ZIPL QC : ---")
print("Total Number of Records :",all_rec)
print("Number of Records with Invalid Zip : ",invalid_zip)
print("NUmber of Records with Invalid Zip and Active Status :",act_invalid_zip)
print("Threshhold check",(act_invalid_zip/all_rec)*100," This value tells us what percentage of active HCPs in the delta are going to whitespace")

--- ZIPL QC : ---
Total Number of Records : 141739
Number of Records with Invalid Zip :  1166
NUmber of Records with Invalid Zip and Active Status : 1023
Threshhold check 0.721749130443985  This value tells us what percentage of active HCPs in the delta are going to whitespace


In [131]:
#CREATE ADDS, UPDATES, AND MERGES DATASETS -
temp_ADDS_UPDATES = pd.DataFrame() #This will hold the New HPCs and Updates
curwk_MERGES = pd.DataFrame() #This will hold the Winner / Looser ID HCPs
enddate_CUST_ENDDATE_CUR_PROC_WK = pd.DataFrame() #This will hold the end date crossed HCPs

#defining conditions - 
cond1 = curwk_CUSTOMER_MASTER['IronwoodWinnerID'].isna() 
cond2 = curwk_CUSTOMER_MASTER['IronwoodLoserID'].isna()
cond3 = curwk_CUSTOMER_MASTER['CustomerEffectiveEndDate'] < TODAY

#applying conditions -
curwk_MERGES = curwk_CUSTOMER_MASTER[(~(cond1) & ~(cond2) & cond3)].copy()
enddate_CUST_ENDDATE_CUR_PROC_WK = curwk_CUSTOMER_MASTER[((cond1 | cond2) & cond3)].copy()
temp_ADDS_UPDATES = curwk_CUSTOMER_MASTER[cond2 & ~(cond3)].copy() #No Value for Winner and Non Endate HCPs

#storing
curwk_MERGES.to_parquet(f's3://{bucket}/{curwk}curwk_MERGES.parquet')
curwk_MERGES.shape

(1134, 70)

In [132]:
#Separating Addtions and Updates

temp_HCP_UPDATES_CUR_PROC_WK = pd.DataFrame() #this will hold updates
temp_HCP_ADDS_CUR_PROC_WK = pd.DataFrame() #this will hold new hcps
temp_b =pd.DataFrame()

MASTER_PROFILE = pd.read_parquet(f's3://{bucket}/{ref_pwek}MASTER_PROFILE.parquet') # fetching previous week's full master profile
ref_pwek_MASTER_PROFILE = pd.DataFrame(MASTER_PROFILE['IID'].copy())
#only keeping IIDs (reason being, if they are update cases , we want all the columns to be repalced)

#merging the datasets , will filter based on indicator (_merge column)
merged_df = pd.merge(ref_pwek_MASTER_PROFILE,temp_ADDS_UPDATES,on='IID',how='outer',indicator=True)

#separating
temp_HCP_UPDATES_CUR_PROC_WK = merged_df[merged_df['_merge'] == 'both']
temp_HCP_UPDATES_CUR_PROC_WK = temp_HCP_UPDATES_CUR_PROC_WK.drop(['_merge'], axis=1).reset_index(drop=True)

temp_b = merged_df[merged_df['_merge'] == 'left_only']
temp_b = temp_b.drop(['_merge'], axis=1).reset_index(drop=True)

temp_HCP_ADDS_CUR_PROC_WK = merged_df[merged_df['_merge'] == 'right_only']
temp_HCP_ADDS_CUR_PROC_WK = temp_HCP_ADDS_CUR_PROC_WK.drop(['_merge'], axis=1).reset_index(drop=True)

#qc -
print(temp_HCP_UPDATES_CUR_PROC_WK.shape)
print(temp_b.shape)
print(temp_HCP_ADDS_CUR_PROC_WK.shape)

#Adding Columns -
adds_HCP_ADDS_CUR_PROC_WK = temp_HCP_ADDS_CUR_PROC_WK.copy() \
.assign(RECORD_START_DATE=DATA_DATE_CALENDAR, RECORD_END_DATE=RECORD_END_DATE)

updates_HCP_UPDATES_CUR_PROC_WK = temp_HCP_UPDATES_CUR_PROC_WK.copy() \
.assign(RECORD_START_DATE=DATA_DATE_CALENDAR, RECORD_END_DATE=RECORD_END_DATE)

(139042, 70)
(1692333, 70)
(1561, 70)


#### CREATE WEEKLY FULL UNIVERSE CUSTOMER MASTER    

In [133]:
temp_CUSTOMER_MASTER_PRE_PROC_WK = pd.DataFrame() #intermediate dataset[ this has all the exclusions applied -END, UPDATE, AND MERGED IID ]

ref_pwek_MASTER_PROFILE = pd.read_parquet(f's3://{bucket}/{ref_pwek}MASTER_PROFILE.parquet') # fetching previous week's full master profile

all_exclusions = pd.DataFrame({
    'IID': pd.concat([
        updates_HCP_UPDATES_CUR_PROC_WK['IID'],
        curwk_MERGES['IID'],
        enddate_CUST_ENDDATE_CUR_PROC_WK['IID']
    ], ignore_index=True).unique()
})

#ALternative Approach , useing isin() method, but it may not be vaible for very large datasets [DOUBT]
#ref_pwek_MASTER_PROFILE = ref_pwek_MASTER_PROFILE[~ref_pwek_MASTER_PROFILE['hub_id'].isin(all_exclusions['hub_id'])]

#going with merge + filter for now as it may be more efficient - 

#merging the datasets , will filter based on indicator (_merge column)
merged_df = pd.merge(ref_pwek_MASTER_PROFILE,all_exclusions,on='IID',how='left',indicator=True)
temp_CUSTOMER_MASTER_PRE_PROC_WK = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)



In [134]:
#qc -
temp_CHECK_MERGE = pd.DataFrame()
ref_pwek_MASTER_PROFILE = pd.DataFrame(ref_pwek_MASTER_PROFILE['IID'].copy())

temp_CHECK_MERGE = pd.merge(ref_pwek_MASTER_PROFILE,(pd.DataFrame(curwk_MERGES['IID'])),on='IID',how='outer',indicator=True)
temp_CHECK_MERGE = temp_CHECK_MERGE[temp_CHECK_MERGE['_merge'] == 'right_only']
print("# of losers found in the previous week master profile : ",len(temp_CHECK_MERGE))
print("Instances inisde mergecass where IID = WinnerID")
df_equal  = curwk_MERGES[curwk_MERGES['IID'] == curwk_MERGES['IronwoodWinnerID']]
df_equal 

# of losers found in the previous week master profile :  1094
Instances inisde mergecass where IID = WinnerID


,IID,CustomerContactType,CustomerStatusCode,IMSID,SecondaryIDSourceName,FirstName,LastName,MiddleName,SuffixName,PrefixName,...,IdentifierType,IdentifierValue,IdentifierStateProvinceCode,IdentifierStatusCode,IdentifierEffectiveStartDate,IdentifierEffectiveEndDate,NPI_ID,NPI_StartDate,NPI_EndDate,TAXONOMYID


In [135]:
temp_CUSTOMER_MASTER_CUR_PROC_WK = pd.DataFrame()

temp_CUSTOMER_MASTER_CUR_PROC_WK = pd.concat([adds_HCP_ADDS_CUR_PROC_WK,
                                              updates_HCP_UPDATES_CUR_PROC_WK,
                                              temp_CUSTOMER_MASTER_PRE_PROC_WK])

# Reset the index
temp_CUSTOMER_MASTER_CUR_PROC_WK = temp_CUSTOMER_MASTER_CUR_PROC_WK.reset_index(drop=True)


In [136]:
#Sort | Drop Dups | Store Data
temp_CUSTOMER_MASTER_CUR_PROC_WK.sort_values('IID',inplace=True)
ref_week_MASTER_PROFILE = temp_CUSTOMER_MASTER_CUR_PROC_WK.drop_duplicates('IID')
num_duplicates_dropped = len(temp_CUSTOMER_MASTER_CUR_PROC_WK) - len(ref_week_MASTER_PROFILE)
print("Number of duplicate records dropped:", num_duplicates_dropped)


#File ready -
MASTER_PROFILE = ref_week_MASTER_PROFILE.copy()
MASTER_PROFILE.to_parquet(f's3://{bucket}/{ref_week}MASTER_PROFILE.parquet')
MASTER_PROFILE.shape

Number of duplicate records dropped: 0


(1832896, 72)

#### Duplicate Specialty Descriptions Check (3.2) -

In [137]:
unique_spec_cds = ref_week_MASTER_PROFILE[['SpecialtyCode', 'SpecialtyDescription']].copy()#get the columns i need for master profile
unique_spec_cds.drop_duplicates(subset='SpecialtyCode', keep='first', inplace=True)#drop drop all duplicates so i only have unique speciality codes
unique_spec_cds.reset_index(drop=True, inplace=True) #reset index
unique_spec_cds_incl = pd.merge(unique_spec_cds,qtrspec_SPEC_INCL_LIN,left_on ='SpecialtyCode',right_on='SPECIALTY_CD',how='left') # joining so i get the spec_incl  flag
# Group by SPECIALTY_DESCRIPTION and count unique spec_incl values
specialty_counts = unique_spec_cds_incl.groupby('SpecialtyDescription')['SPEC_INCL'].nunique()
invalid_specialties = specialty_counts[specialty_counts > 1].index
# Display the invalid specialties
print("Invalid Specialties:",invalid_specialties)
qc_df_print = unique_spec_cds_incl[unique_spec_cds_incl['SpecialtyDescription'].isin(invalid_specialties)].copy()
qc_df_print.sort_values(by='SpecialtyDescription',inplace=True)
qc_df_print

Invalid Specialties: Index(['ABDOMINAL', 'CARDIOVASC', 'CERTIFIED', 'INTERNAL M', 'PEDIATRIC'], dtype='object', name='SpecialtyDescription')


,SpecialtyCode,SpecialtyDescription,SPECIALTY_CD,SPECIALTY_DESCRIPTION,SPEC_INCL
1,AS,ABDOMINAL,AS,ABDOMINAL SURGERY,N
357,AR,ABDOMINAL,AR,ABDOMINAL RADIOLOGY,Y
6,CD,CARDIOVASC,CD,CARDIOVASCULAR DISEASE,Y
246,CDS,CARDIOVASC,CDS,CARDIOVASCULAR SURGERY,N
316,CNM,CERTIFIED,CNM,CERTIFIED NURSE MIDWIFE,Y
317,CNA,CERTIFIED,CNA,CERTIFIED NURSE ANESTHETIST,N
247,MP,INTERNAL M,MP,INTERNAL MEDICINE-PSYCHIATRY,Y
248,IFP,INTERNAL M,IFP,INTERNAL MEDICINE/FAMILY PRACTICE,Y
288,MPM,INTERNAL M,MPM,INTERNAL MEDICINE/REHABILITAION,N
342,IPM,INTERNAL M,IPM,INTERNAL MEDICINE-PREVENTIVE MEDICINE,Y


#### REFERENCE DATE PARAMETERS DATA SETS CREATION  -

In [138]:
data = {
    'DATA': ['XPONENT', 'CALLS', 'EVENTS', 'MASTER PROFILE'],
    'CALENDAR': [DATA_DATE_CALENDAR] * 4,
    'DATADATE': [DATA_DATE_WEEKLY,EVTS_DATE_WEEKLY,EVTS_DATE_WEEKLY,DATA_DATE_CALENDAR]
}

ref_week_DATE_PARM = pd.DataFrame(data)
ref_week_DATE_PARM

,DATA,CALENDAR,DATADATE
0,XPONENT,2024-07-01,2024-06-14
1,CALLS,2024-07-01,2024-06-28
2,EVENTS,2024-07-01,2024-06-28
3,MASTER PROFILE,2024-07-01,2024-07-01


In [139]:
data = {
    'DATA': ['XPONENT', 'CALLS', 'EVENTS', 'MASTER PROFILE'],
    'CALENDAR': [DATA_DATE_CALENDAR] * 4,
    'DATADATE': [DATA_DATE_MONTHLY,DATA_DATE_MONTHLY,DATA_DATE_MONTHLY,DATA_DATE_CALENDAR]
}

ref_mnth_DATE_PARM = pd.DataFrame(data)
ref_mnth_DATE_PARM

,DATA,CALENDAR,DATADATE
0,XPONENT,2024-07-01,2024-06-14
1,CALLS,2024-07-01,2024-06-14
2,EVENTS,2024-07-01,2024-06-14
3,MASTER PROFILE,2024-07-01,2024-07-01


In [140]:
ref_week_DATE_PARM.to_parquet(f's3://{bucket}/{ref_week}DATE_PARM.parquet')
ref_mnth_DATE_PARM.to_parquet(f's3://{bucket}/{ref_month}DATE_PARM.parquet')

In [141]:
#DELETEING non used Dfs?
del temp_CUSTOMER_MASTER_CUR_PROC_WK
del ref_week_MASTER_PROFILE
del merged_df
del temp_CUSTOMER_MASTER_PRE_PROC_WK
del temp_ADDRESS1_F
del temp_ADDRESS1
del temp_test1
del temp_ADDRESS
del curwk_ADDRESS
del temp_b
del curwk_CUSTOMER_MASTER
del updates_HCP_UPDATES_CUR_PROC_WK
del temp_ADDS_UPDATES
del temp_HCP_UPDATES_CUR_PROC_WK
del temp_ADDRESSFLAG1
del curwk_COMMUNICATION
del temp_COMMUNICATION1
del temp_CUSTOMER1
del temp_CUSTOMER
del curwk_CUSTOMER
del curwk_DEA
del temp_Addr_all
del temp_DEA1
del temp_ADDRESSNODUP
del temp_test4
gc.collect()

15004

### Net New HCPs

In [142]:
# Current Week Full Master Profile - MASTER_PROFILE (Already Loaded in memory)
CUR_PROFILE = pd.read_parquet(f's3://{bucket}/{ref_week}MASTER_PROFILE.parquet')
# Previous Week Master Profile
PRE_PROFILE = pd.read_parquet(f's3://{bucket}/{ref_pwek}MASTER_PROFILE.parquet')

# Perform the merge using pd.merge
NET_NEW = pd.merge(CUR_PROFILE, PRE_PROFILE['IID'],on='IID', how='left', indicator=True)
NET_NEW = NET_NEW[NET_NEW['_merge'] == 'left_only']
NET_NEW = NET_NEW.drop(['_merge'], axis=1).reset_index(drop=True)

#adding week timestamp 
NET_NEW['SOURCE'] = CUR_WK  #using cur_wk_xpn_date but in code  -"&CUR_WK." is used.

### Appending net new to create rolling file
- the week after frozen week , the net new file is reset
- otherwise, net new files gets accumilated by using previous week net new

In [143]:
#if WEEK1 == 1 then dont append to previous week net new , else append.
WEEK1 = 0 #DONT CHANGE THIS  , You have to manually reset net net for a new Qtr in a separate code , reuse the code there
NET_NEW_ACCU = pd.DataFrame()

net_new_varname = "NET_NEW_{}".format(QTR_NET)

if WEEK1 == 1:
    NET_NEW_ACCU = NET_NEW.copy()
elif WEEK1 == 0:
    globals()[net_new_varname] = pd.read_parquet(f's3://{bucket}/{ref_pwek}{net_new_varname}.parquet')
    columns_to_drop = ['Territory_IW1', 'Territory_Name_IW1','Region', 
                   'Region_Name', 'Area','Area_Name','SPECIALTY_DESCRIPTION',
                   'SPEC_INCL_LIN','IC_INCL_LIN'] 
    globals()[net_new_varname] = globals()[net_new_varname].drop(columns=columns_to_drop)
    NET_NEW_ACCU = pd.concat([NET_NEW, globals()[net_new_varname]], axis=0)

#### Assigning Terr information from Zip to Terr
- Addtionally, using the quarterly Speciality Inclusion Exclusion list to get inclusion status

In [144]:
ZIP_TO_TERR = pd.read_parquet(f's3://{bucket}/{ZIP}zip_to_terr.parquet')
ZIP_TO_TERR.rename(columns={'Territory':'Territory_IW1','Territory_Name':'Territory_Name_IW1'},inplace=True) #DOUBT not sure why we do this
SPEC_INCL_LIN = pd.read_parquet(f's3://{bucket}/{qtrspec}qtrspec_SPEC_INCL_LIN.parquet')
#Extra
SPEC_INCL_LIN.rename(columns={'SPECIALTY_CD':'SpecialtyCode'},inplace=True)
SPEC_INCL_LIN.set_index('SpecialtyCode', inplace=True)

In [145]:
#getting terr name, region name, area name from zip to terr
NET_NEW_FN = pd.merge(NET_NEW_ACCU,ZIP_TO_TERR,left_on='ZIP',right_on='Zip',how='left')
whitespace_values = {
    'Territory_IW1': "1111-99999-99",
    'Territory_Name_IW1': "",
    'Region': "1111-999",
    'Region_Name': "",
    'Area': "1111-9",
    'Area_Name': ""
}
# Fill whitespace_values with the default values
NET_NEW_FN.fillna(whitespace_values, inplace=True)

#getting speciality flags from spec incl lin quaterly file -[NOTE : BUG FIX SUMMARY - sas did not perform a left join, revised code to perfrom mapping strategy] 
NET_NEW_FN['SPECIALTY_DESCRIPTION'] = NET_NEW_FN['SpecialtyCode'].map(SPEC_INCL_LIN['SPECIALTY_DESCRIPTION'])
NET_NEW_FN['SPEC_INCL_LIN'] = NET_NEW_FN['SpecialtyCode'].map(SPEC_INCL_LIN['SPEC_INCL'])
NET_NEW_FN.fillna({'SPECIALTY_DESCRIPTION':''},inplace=True)

#adding new column SPEC_INCL_LIN (this is based on match code, so ill have to wait for it to come.- NOTE  Condition is only applied on failed left matches
#NET_NEW_FN['SPEC_INCL_LIN'] = ['N' if x == '01' else 'Y' for x in NET_NEW_FN['MatchCode']] - This is wrong
NET_NEW_FN['SPEC_INCL_LIN'] = np.where(NET_NEW_FN['SPEC_INCL_LIN'].isna(),
                                        np.where(NET_NEW_FN['MatchCode'] == '01', 'N', 'Y'),
                                        NET_NEW_FN['SPEC_INCL_LIN'])

#dropping extra columns (different name , same data)
NET_NEW_FN.drop(['Zip'], axis=1, inplace=True)

### Filtering for any legal removal and unknown address HCPs
- 'Jami' file is used here (double check import location and check for file updates on quarter change)

In [146]:
# applying exclusions
JAMI_INCLEXCL = pd.read_parquet(f's3://{bucket}/{jami}jami_inclexcl.parquet')
jami_exclusions = pd.DataFrame()
jami_exclusions = JAMI_INCLEXCL[JAMI_INCLEXCL['TYPE'].isin(['Legal Removals', 'Unknown Address'])]
jami_exclusions = jami_exclusions[['IID']].copy()
jami_exclusions.drop_duplicates(subset='IID', inplace=True)
jami_exclusions.reset_index(drop=True, inplace=True)

ref_week_NET_NEW = pd.merge(NET_NEW_FN,jami_exclusions,on = 'IID',how='left',indicator=True)
ref_week_NET_NEW = ref_week_NET_NEW[ref_week_NET_NEW['_merge'] == 'left_only']
ref_week_NET_NEW = ref_week_NET_NEW.drop(['_merge'], axis=1).reset_index(drop=True)

#### IC flag creation
- IC_INCL_LIN is a an important flag which decides if a hcp should be considered for IC or not
- CustomerEffectiveStartDate , SPEC_INCL_LIN, CustomerStatusCode, MatchCode decide its value

In [147]:
ref_week_NET_NEW['IC_INCL_LIN'] = 0  # Initialize the column with 0

condition = (
    (ref_week_NET_NEW['CustomerEffectiveStartDate'] < CUT_OFF_net) &
    (ref_week_NET_NEW['SPEC_INCL_LIN'] == "Y") &
    (ref_week_NET_NEW['CustomerStatusCode'] == "Active") &
    (ref_week_NET_NEW['MatchCode'] != '01')
)

ref_week_NET_NEW.loc[condition, 'IC_INCL_LIN'] = 1
print(ref_week_NET_NEW.shape)


(43433, 82)


#### Importing IIDs from frozen IC universe , and deleting any occurances of those in net new as an added QC 

In [148]:
CUSTOMER_MASTER_IC_LIN = pd.read_parquet(f's3://{bucket}/{qtrspec}CUSTOMER_MASTER_IC_LIN.parquet')
CUSTOMER_MASTER_IC_LIN = pd.DataFrame(CUSTOMER_MASTER_IC_LIN['IID'])
#Faulty Approach - needs checking into : Try using - ref_week_NET_NEW = merged_df[merged_df['_merge'] == 'left_only']
# incase of memory issues ?
# merged_df = pd.merge(ref_week_NET_NEW, CUSTOMER_MASTER_IC_LIN[['IID']], on='IID', how='inner')
# ref_week_NET_NEW = ref_week_NET_NEW.drop(merged_df.index)
#updated approach - 
qc_bef = len(ref_week_NET_NEW)
ref_week_NET_NEW = ref_week_NET_NEW[~ref_week_NET_NEW['IID'].isin(CUSTOMER_MASTER_IC_LIN['IID'])]
qc_af = len(ref_week_NET_NEW)
print("Records Dropped from ic uni: ",qc_bef-qc_af)

Records Dropped from ic uni:  0


In [149]:
#Only Keep the first Instance of IID Source.
qc_bef = len(ref_week_NET_NEW)
ref_week_NET_NEW.sort_values(by=['IID', 'SOURCE'], inplace=True)
ref_week_NET_NEW.drop_duplicates(subset='IID', keep='first', inplace=True)
qc_af = len(ref_week_NET_NEW)
print("Records Dropped : ",qc_bef-qc_af)

ref_week_NET_NEW = ref_week_NET_NEW.sort_values(['IID'])
ref_week_NET_NEW.reset_index(drop=True, inplace=True)

#sdap here
ref_week_NET_NEW.to_parquet(f's3://{bucket}/{ref_week}{net_new_varname}.parquet')

Records Dropped :  0


#### QC | Frequency Table to QC number of records , OC flags , date of addition etc
- Carry over from sas

In [150]:
#17.1: Check current week's net_new source
# Perform the frequency count
freq_counts = ref_week_NET_NEW['SOURCE'].value_counts().reset_index()

# Rename the columns
freq_counts.columns = ['SOURCE', 'COUNT']

# Sort the frequencies in descending order
freq_counts = freq_counts.sort_values('SOURCE')

# Print the frequency counts
print(freq_counts)

      SOURCE  COUNT
3   20240202   2496
10  20240209   1914
8   20240216   2041
12  20240223   1868
5   20240301   2236
6   20240308   2184
9   20240315   1927
7   20240322   2155
1   20240329   3148
0   20240405   5682
4   20240412   2297
13  20240419   1654
2   20240426   2743
18  20240503   1503
17  20240510   1523
14  20240517   1650
15  20240524   1626
11  20240531   1895
19  20240607   1330
16  20240614   1561


In [151]:
#17.2: Check IC_INCL_LIN - this wont work yet because flag is not there (match code not avilable)
freq_counts = ref_week_NET_NEW['IC_INCL_LIN'].value_counts().reset_index()

# Rename the columns
freq_counts.columns = ['IC_INCL_LIN', 'COUNT']

# Sort the frequencies in descending order
freq_counts = freq_counts.sort_values('COUNT', ascending=False)

# Print the frequency counts #NOTE : last check on 27-12-2023 , 16 people with IC flag 0 are lesser in python compared to sas presumably Jami ppl
print(freq_counts)

   IC_INCL_LIN  COUNT
0            1  27092
1            0  16341


In [152]:
# 17.3: Check Net New for Dups
print("hcp_count",ref_week_NET_NEW['IID'].count())
print("distinct_hcp_count",ref_week_NET_NEW['IID'].nunique())

hcp_count 43433
distinct_hcp_count 43433


In [153]:
# TO DO/
# Check for Net New NOBS COUNT - DONE
# Impliment SAS method to only keep Oldest Record - DONE
# Apply QC that checks against frozen MP - DONE
# Export Dataframe - DONE
# Remove Redundant Cells and Time - DONE
# check why speciality rank has 2 columns